In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sb

In [3]:
housing_data = pd.read_csv('DatenAusgegeben1.2_UTF8_manuell.csv', sep=";", keep_default_na=False)

# Bereinigung und Feature Engeneering

# für €/qm² für besseren Vergleich der Daten
housing_data["Preisproqm"] = housing_data["Preis"]/housing_data["Wohnflaeche in qm"]

# monate seit 2136
housing_data["Monate"] = (housing_data["Verkaufsjahr"]-2136)*12 + housing_data["Verkaufsmonat"] -1

# Werte der Heizungsqualitaet in Zahlen umwandeln
housing_data["Heizungsqualitaet"] = [1 if x=="Schl" else 2 if x=="Ud" else 3 if x=="Ty" else 4 if x=="Gut" else 5 for x in housing_data["Heizungsqualitaet"]]

# Zustand, 1-10 wird auf 1-5 gemapt, das ist zwar ein bisschen ungenauer, aber einfacher zu vergleichen
housing_data["Zustandf"] = [int(x/2) + x%2 for x in housing_data["Zustand"]]

# Alle Zustände werden von 1...X auf 0....X-1 geschoben
housing_data["HeizungsqualitaetN"] = [ x-1 for x in housing_data["Heizungsqualitaet"]]
housing_data["ZustandN"] = [ x-1 for x in housing_data["Zustandf"]]
housing_data["FassadeN"] = [ x-1 for x in housing_data["Zustand Fassade"]]
housing_data["KuecheN"] = [ x-1 for x in housing_data["Kuechenqualitaet"]]

# NA in Garage Typ wird zu "Keine Garage"
housing_data["Garage Typ"] = ['keine Garage' if x=='NA' else x for x in housing_data["Garage Typ"]]

In [4]:
#Extract X and y from dataset

X = housing_data.iloc[:, :27].to_numpy()
y = housing_data["Preis"].to_numpy()

print(X.shape, y.shape)

(2000, 27) (2000,)


In [5]:
#Prepare data

#Most important features renovation?, reselltime?, air conditioning, heater quality, kitchen quality, state of house, state of facade 

#Encode features (Label ord One-Hot)

#Separate into training- and testdata
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(1600, 27) (1600,) (400, 27) (400,)


In [10]:
X_train

array([[892, 'Reg', 'Nein', ..., 'NA', 5, 2137],
       [1214, 'Reg', 'Nein', ..., 'NA', 11, 2138],
       [1334, 'IR1', 'Ja', ..., 'NA', 4, 2137],
       ...,
       [1174, 'IR1', 'Nein', ..., 'NA', 7, 2136],
       [869, 'Reg', 'Ja', ..., 'NA', 6, 2140],
       [1017, 'Reg', 'Nein', ..., 'NA', 5, 2140]], dtype=object)

In [6]:
 #Regression
from sklearn import linear_model
reg = linear_model.LinearRegression()
reg.fit(X_train[0], y_train)

ValueError: could not convert string to float: 'Reg'

In [26]:
#Validation
y_pred = reg.predict(X_test["Grundstueck in qm"].to_numpy().reshape(-1, 1))

reg.score(X_test["Grundstueck in qm"].to_numpy().reshape(-1, 1), y_pred)

1.0

In [29]:
X_test["Grundstueck in qm"].to_numpy()

array([ 828, 1344,  758,  212, 1177, 1150,  780,  887,  802,  956, 1533,
       2368, 1240,  799,  836,  426, 1096, 1649,  343,  725,  557,  549,
        911,  818, 1055,  716,  922, 1057, 1317,  156,  896,  201,  805,
        669,  975,  495,  142,  906,  815, 1025,  968, 1085,  475, 1317,
        860, 1045,  802,  927,  296, 1263,  892, 1000, 1214, 4367,  545,
       1549, 2349,  465, 2023,  836,  954, 1447, 1093,  682,  671,  733,
        285,  557,  641,  624,  920, 1371,  892,  697, 1630,  520, 1314,
        966,  974,  720,  596,  785,  457,  892,  728,  943,  946,  425,
        858,  891,  248,  886,  359,  818, 1268, 1066,  372, 1166,  434,
        465,  354, 1051,  996,  743,  669,  580, 1178,  950,  623,  973,
        582,  903,  895,  972,  298,  862, 1320, 1093, 1985,  674, 1145,
        782,  148,  951, 1076,  969,  538,  664,  569,  718, 1044, 1027,
        502,  142,  142,  907, 1326, 2429, 1547,  728, 1429, 1095,  589,
       1307,  744, 1241, 1322, 1234,  341, 1051, 10